In [1]:
import pandas as pd
import random
import string
from faker import Faker

# Create a Faker instance
fake = Faker()

# Define the number of rows you want in your dataset
user_num_rows = 1000  # You can change this number as needed

# Generate random data
data = {
     'userId': list(range(1, user_num_rows + 1)),
    'userName': [fake.name() for _ in range(user_num_rows)],
    'age': [random.randint(18, 65) for _ in range(user_num_rows)],
    'gender': [random.choice(['M', 'F']) for _ in range(user_num_rows)],
}

# Create a DataFrame
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
df.to_csv('random_user_data.csv', index=False)

In [2]:

# Define the number of rows you want in your dataset
service_num_rows = 1500  # You can change this number as needed

# Generate random data for the "service" table
data = {
    'serviceId': list(range(1, service_num_rows + 1)),
    'serviceName': [fake.word().capitalize() for _ in range(service_num_rows)],
}

# Create a DataFrame for the "service" table
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
df.to_csv('random_service_data.csv', index=False)


In [3]:
# Define the number of rows you want in your "ratings" dataset
ratings_num_rows = 100000  # You can change this number as needed

# Create a DataFrame for the "ratings" table
ratings_data = {
    'userId': [random.randint(1, user_num_rows) for _ in range(ratings_num_rows)],
    'serviceId': [random.randint(1, service_num_rows) for _ in range(ratings_num_rows)],
    'rating': [random.randint(1, 5) for _ in range(ratings_num_rows)],
}

ratings_df = pd.DataFrame(ratings_data)

# Save the DataFrame to a CSV file
ratings_df.to_csv('random_ratings_data.csv', index=False)

In [4]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.sql.functions import col
from recommenders.utils.timer import Timer
from recommenders.evaluation.spark_evaluation import SparkRatingEvaluation, SparkRankingEvaluation
from recommenders.datasets.spark_splitters import spark_random_split
from recommenders.utils.notebook_utils import is_jupyter
from pyspark.sql.functions import col, collect_list,pow,sqrt,abs

In [5]:
spark = SparkSession.builder.appName("ServiceRecommendation").getOrCreate()

23/09/06 15:48:02 WARN Utils: Your hostname, razin-ThinkPad resolves to a loopback address: 127.0.1.1; using 10.10.20.157 instead (on interface wlp0s20f3)
23/09/06 15:48:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/06 15:48:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
# Read the user data CSV file into a DataFrame
user_df = spark.read.csv('random_user_data.csv', header=True, inferSchema=True)

# Read the service data CSV file into a DataFrame
service_df = spark.read.csv('random_service_data.csv', header=True, inferSchema=True)

# Read the ratings data CSV file into a DataFrame
ratings_df = spark.read.csv('random_ratings_data.csv', header=True, inferSchema=True)


In [7]:
user_df.show()
service_df.show()
ratings_df.show()

+------+---------------+---+------+
|userId|       userName|age|gender|
+------+---------------+---+------+
|     1|     Tammy Bond| 27|     M|
|     2|   Anthony Ward| 46|     M|
|     3|      Paul Ware| 39|     F|
|     4|   Dustin Smith| 53|     F|
|     5|   Sean Mcgrath| 62|     M|
|     6|   Peter Warren| 43|     F|
|     7|      Seth Hall| 23|     F|
|     8|    Seth Krause| 61|     F|
|     9|  Marco Andrews| 22|     F|
|    10|    Collin Huff| 50|     F|
|    11|   Jennifer Lam| 33|     M|
|    12|     Lori Brown| 45|     F|
|    13|   Sylvia Klein| 40|     M|
|    14|    Joshua Cook| 28|     M|
|    15|  Anthony Ellis| 58|     F|
|    16|    Kelsey Hahn| 28|     F|
|    17|Zachary Elliott| 45|     F|
|    18| Angela Stewart| 21|     F|
|    19|   Keith Hodges| 34|     M|
|    20| Melissa Adkins| 63|     M|
+------+---------------+---+------+
only showing top 20 rows

+---------+-----------+
|serviceId|serviceName|
+---------+-----------+
|        1|      Court|
|        2|   

In [8]:
# Join ratings, users, and movies dataframes
joined_df = ratings_df.join(user_df.select("userId", "gender", "age"), on="userId", how="inner")
joined_df = joined_df.filter(col("serviceId").isNotNull())
joined_df.show()

+------+---------+------+------+---+
|userId|serviceId|rating|gender|age|
+------+---------+------+------+---+
|   706|      508|     1|     F| 21|
|   429|      888|     4|     M| 40|
|   329|      911|     2|     F| 40|
|    60|      838|     4|     M| 37|
|   447|     1396|     3|     F| 37|
|   680|      817|     5|     F| 59|
|   717|     1031|     3|     F| 22|
|   718|      807|     4|     M| 39|
|   165|     1206|     3|     F| 51|
|   588|      908|     4|     F| 57|
|   483|      696|     3|     M| 27|
|   537|     1406|     4|     F| 62|
|   227|      566|     3|     M| 41|
|    42|      174|     5|     M| 37|
|   817|      651|     3|     M| 59|
|   606|      986|     5|     F| 65|
|   630|     1339|     3|     F| 25|
|   199|     1179|     4|     F| 27|
|   230|      455|     2|     M| 65|
|   401|     1167|     5|     M| 24|
+------+---------+------+------+---+
only showing top 20 rows



In [9]:
# Define gender, age, and occupation filters
#gender_filter = "F"
#age_filter_min = 18
#age_filter_max = 24
#occupation_filter = "writer"

# Filter data based on gender, age, and occupation
#filtered_df = joined_df.filter(
#    (col("gender") == gender_filter) &
#    (col("age") <= age_filter_max) &
#    (col("age") >= age_filter_min)
#)

#filtered_df.show()

In [10]:
train, test = spark_random_split(joined_df, ratio=0.75, seed=123)
print("N train", train.cache().count())
print("N test", test.cache().count())

N train 75018
N test 24982


In [11]:
COL_USER = "userId"
COL_ITEM = "serviceId"
COL_RATING = "rating"

# Train the ALS model
als = ALS(
    rank=10,
    maxIter=15,
    implicitPrefs=False,
    regParam=0.05,
    coldStartStrategy='drop',
    nonnegative=False,
    seed=42,
    userCol=COL_USER, itemCol=COL_ITEM, ratingCol=COL_RATING
)

In [12]:
with Timer() as train_time:
    model = als.fit(train)

print("Took {} seconds for training.".format(train_time.interval))

23/09/06 15:48:11 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/09/06 15:48:11 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


Took 3.9263531800024793 seconds for training.


In [13]:
with Timer() as test_time:

    # Get the cross join of all user-item pairs and score them.
    users = train.select(COL_USER).distinct()
    items = train.select(COL_ITEM).distinct()
    user_item = users.crossJoin(items)
    dfs_pred = model.transform(user_item)


        # Remove seen items.
    dfs_pred_exclude_train = dfs_pred.alias("pred").join(
        train.alias("train"),
        (col("pred." + COL_USER) == col("train." + COL_USER)) &
        (col("pred." + COL_ITEM) == col("train." + COL_ITEM)),
        how='outer'
    )
    
    top_all = dfs_pred_exclude_train.filter(dfs_pred_exclude_train[f"train.{COL_RATING}"].isNull()) \
        .select('pred.' + COL_USER, 'pred.' + COL_ITEM, 'pred.' + "prediction")

    # In Spark, transformations are lazy evaluation
    # Use an action to force execute and measure the test time 
    top_all.cache().count()

print("Took {} seconds for prediction.".format(test_time.interval))
top_all.show()

Took 10.82458040399797 seconds for prediction.
+------+---------+----------+
|userId|serviceId|prediction|
+------+---------+----------+
|     1|      587| 2.1268969|
|     1|      869| 3.4434083|
|     1|     1208| 5.3751073|
|     1|     1348| 3.7945688|
|     1|     1357| 1.9100304|
|     2|       80| 1.9099348|
|     2|      472| 0.7585122|
|     2|      582| 3.9361854|
|     2|      838| 1.4924681|
|     2|      975| 0.8307763|
|     2|     1260| 2.3388429|
|     2|     1325|  1.615089|
|     2|     1381| 4.2416854|
|     3|       22| 2.8967817|
|     3|       57| 2.5475547|
|     3|       89| 2.0668979|
|     3|      367| 3.5179217|
|     3|     1091| 2.0814824|
|     3|     1167| 4.2013016|
|     3|     1499| 3.7177808|
+------+---------+----------+
only showing top 20 rows



In [16]:
# Generate predicted ratings.
prediction = model.transform(test)
prediction.cache().show()

+------+---------+------+------+---+----------+
|userId|serviceId|rating|gender|age|prediction|
+------+---------+------+------+---+----------+
|   633|      148|     3|     F| 41| 2.2394624|
|   633|      148|     3|     F| 41| 2.2394624|
|   101|      148|     5|     F| 49| 4.6426487|
|   346|      148|     1|     F| 21|  4.050703|
|   519|      148|     2|     F| 25|  5.027049|
|   696|      148|     5|     M| 57| 2.4729042|
|   386|      148|     3|     M| 46|  2.767638|
|   435|      148|     2|     M| 51| 3.2766035|
|   572|      148|     2|     F| 59| 2.3134737|
|    64|      148|     1|     M| 33|   3.49406|
|   886|      148|     3|     F| 32| 4.3128533|
|   712|      148|     1|     M| 41| 2.6545491|
|   243|      463|     1|     M| 64| 1.9968162|
|   513|      463|     1|     M| 48| 3.9708512|
|   597|      463|     5|     M| 62| 3.8436112|
|   368|      463|     5|     M| 40| 2.9039812|
|   388|      463|     1|     F| 58| 2.3138795|
|   409|      463|     1|     F| 56|  3.

In [28]:
from pyspark.sql.functions import sum ,mean,avg
prediction_with_accuracy = prediction.withColumn("accuracy", abs(col("rating") - col("prediction")))


# Show the DataFrame with the "accuracy" column
prediction_with_accuracy.show()

#This code calculates the MAE between the "rating" and "prediction" columns and then converts it into accuracy by subtracting it from 1.0 and multiplying by 100 to get a percentage value. It will print both the MAE and accuracy.



+------+---------+------+------+---+----------+----------+
|userId|serviceId|rating|gender|age|prediction|  accuracy|
+------+---------+------+------+---+----------+----------+
|   633|      148|     3|     F| 41| 2.2394624| 0.7605376|
|   633|      148|     3|     F| 41| 2.2394624| 0.7605376|
|   101|      148|     5|     F| 49| 4.6426487| 0.3573513|
|   346|      148|     1|     F| 21|  4.050703|  3.050703|
|   519|      148|     2|     F| 25|  5.027049|  3.027049|
|   696|      148|     5|     M| 57| 2.4729042| 2.5270958|
|   386|      148|     3|     M| 46|  2.767638|0.23236203|
|   435|      148|     2|     M| 51| 3.2766035| 1.2766035|
|   572|      148|     2|     F| 59| 2.3134737| 0.3134737|
|    64|      148|     1|     M| 33|   3.49406|   2.49406|
|   886|      148|     3|     F| 32| 4.3128533| 1.3128533|
|   712|      148|     1|     M| 41| 2.6545491| 1.6545491|
|   243|      463|     1|     M| 64| 1.9968162|0.99681616|
|   513|      463|     1|     M| 48| 3.9708512| 2.970851

In [32]:
overall_accuracy = prediction_with_accuracy.agg({"accuracy": "mean"}).collect()[0][0]
percentage_accuracy = (1-(overall_accuracy/5))*100
# Show the overall accuracy
print(f"Percentage_accuracy: {percentage_accuracy:.2f}")

Percentage_accuracy: 71.27


In [ ]:
# Cleanup Spark instance
spark.stop()